# Named Entity Recognition using Language Models

This lab will focus on helping you fine-tune Transformers models for Named Entity Recognition. You will use the Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

## Installation
Let us first install the required libraries

In [1]:
!pip install datasets transformers seqeval huggingface_hub

     |████████████████████████████████| 325 kB 4.2 MB/s 
     |████████████████████████████████| 3.8 MB 49.5 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 136 kB 50.6 MB/s 
     |████████████████████████████████| 212 kB 42.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.0 MB/s 
     |████████████████████████████████| 127 kB 38.2 MB/s 
     |████████████████████████████████| 6.5 MB 40.5 MB/s 
     |████████████████████████████████| 596 kB 53.3 MB/s 
     |████████████████████████████████| 895 kB 24.5 MB/s 
     |████████████████████████████████| 271 kB 52.0 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 50.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=5d5176e368ba066724b3efd7f0d247cba1ac3b2721f834500dce3f22792843c4
  Stored in directory: /root/.cache/pip/whe

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 82 not upgraded.
Need to get 6,800 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.1.2 [6,800 kB]
Fetched 6,800 kB in 1s (6,608 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/per

## Dataset(s)

We will use the Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be done with the functions load_dataset and load_metric.

The notebook should work with any token classification dataset provided by the Datasets library.

In [3]:
from datasets import load_dataset, load_metric, concatenate_datasets
datasets = load_dataset("Babelscape/wikineural")

Using custom data configuration Babelscape--wikineural-2c05bc228ce4015c


Extracting data files:   0%|          | 0/27 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/Babelscape--wikineural-2c05bc228ce4015c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/27 [00:00<?, ?it/s]

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

In [4]:
datasets["train_en"][0]

{'lang': 'en',
 'ner_tags': [0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0],
 'tokens': ['This',
  'division',
  'also',
  'contains',
  'the',
  'Ventana',
  'Wilderness',
  ',',
  'home',
  'to',
  'the',
  'California',
  'condor',
  '.']}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [5]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

If you want to train a monolingual model (NER on a single language), you have to use a single language for each split.

In [6]:
#train_dataset = datasets["train_en"]
#val_dataset = datasets["val_en"]
#test_dataset = datasets["test_en"]

This is how you can concatenate multiple dataset splits to train a Multilingual NER model. However, let us focus on Monolingual NER for now.

In [26]:
train_dataset = concatenate_datasets( [ datasets["train_de"], datasets["train_es"] ] )
#                                       datasets["train_en"], 
#                                       datasets["train_fr"],
#                                       datasets["train_it"],
#                                       datasets["train_nl"],
#                                       datasets["train_pl"],
#                                       datasets["train_pt"],
#                                       datasets["train_ru"]])

val_dataset = concatenate_datasets( [datasets["val_de"], datasets["val_es"] ] )
#                                     datasets["val_en"], 
#                                     datasets["val_fr"],
#                                     datasets["val_it"],
#                                     datasets["val_nl"],
#                                     datasets["val_pl"],
#                                     datasets["val_pt"],
#                                     datasets["val_ru"]])

test_dataset = concatenate_datasets([datasets["test_de"], datasets["test_es"] ] ) 
#                                     datasets["test_en"], 
#                                     datasets["test_fr"],
#                                     datasets["test_it"],
#                                     datasets["test_nl"],
#                                     datasets["test_pl"],
#                                     datasets["test_pt"],
#                                     datasets["test_ru"]])

## Pre-processing

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

 - we get a tokenizer that corresponds to the model architecture we want to use,
 - we download the vocabulary used when pretraining this specific checkpoint.
 
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [27]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") ## We use the BERT tokenizer

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [28]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can directly call this tokenizer on one sentence, or you can use the tokenize method to tokenize multiple sentences at once.

In [29]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

This is a good overal tutorial for preprocessing for Transformer. Check this out to learn more about the preprocessing steps.
https://huggingface.co/transformers/preprocessing.html

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Just change the value of the following flag:

In [30]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [31]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [32]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/175 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning

In [33]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 5,
    "B-MISC": 7,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-MISC": 8,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_p

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [34]:
model_name = "bert-base-multilingual-cased" ## You Change this to any generic language model from HuggingFace which supports fine-tuning.
args = TrainingArguments(                   ## This is where all your hyperparameters are, reduce *_batch_size if you are working with a GPU which has limited memory.
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,                     ## may not be enough for a large data, increase this to 5 later in your experiments.
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [35]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [36]:
metric = load_metric("seqeval")

So we will need to do a bit of post-processing on our predictions:

 - select the predicted index (with the maximum logit) for each token
 - convert it to its string label
 - ignore everywhere we set a label of -100
 
The following function does all this post-processing on the result of Trainer.evaluate (which is a namedtuple containing predictions and labels) before applying the metric:

In [37]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Before you go any further:
1. Login to the HuggigFace CLI on your system using command ``huggingface-cli login``
2. Get your access token from your user account on HuggingFace at https://huggingface.co/settings/tokens (preferably a write access token)
3. Use this token with the command above to ensure being logged in to HuggingFace.

IF you are not working on your own machine with a GPU and have opened this notebook in Colab / Studio Lab, you can use the also cell below to login. You could have used this cell instead of commandline (cli) login, but where is the fun in that?

In [38]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the Trainer:

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/wikineural-multilingual-ner is already a clone of https://huggingface.co/VikasMani/wikineural-multilingual-ner. Make sure you pull the latest changes with `repo.git_pull()`.


In [40]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 174960
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10935


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
10000,0.020900,0.019031,0.958080,0.963768,0.960916,0.994242


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21990
  Batch size = 16
Saving model checkpoint to wikineural-multilingual-ner/checkpoint-10000
Configuration saved in wikineural-multilingual-ner/checkpoint-10000/config.json
Model weights saved in wikineural-multilingual-ner/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in wikineural-multilingual-ner/checkpoint-10000/tokenizer_config.json
Special tokens file saved in wikineural-multilingual-ner/checkpoint-10000/special_tokens_map.json
tokenizer config file saved in wikineural-multilingual-ner/tokenizer_config.json
Special tokens file saved in wikineural-multilingual-ner/special_tokens_map.json


Training completed. Do not forget to shar

TrainOutput(global_step=10935, training_loss=0.03832499927508542, metrics={'train_runtime': 6659.7849, 'train_samples_per_second': 26.271, 'train_steps_per_second': 1.642, 'total_flos': 8557250914209312.0, 'train_loss': 0.03832499927508542, 'epoch': 1.0})

## Evaluation

The evaluate method allows you to evaluate again on the evaluation dataset or on another dataset:

In [41]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21990
  Batch size = 16


{'epoch': 1.0,
 'eval_accuracy': 0.9943252485807478,
 'eval_f1': 0.9605338995904747,
 'eval_loss': 0.01865452341735363,
 'eval_precision': 0.9570210965362993,
 'eval_recall': 0.9640725855559615,
 'eval_runtime': 267.2823,
 'eval_samples_per_second': 82.273,
 'eval_steps_per_second': 5.144}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the predict method:

In [42]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 21990
  Batch size = 16


{'LOC': {'f1': 0.9691744102930665,
  'number': 11157,
  'precision': 0.9661530239600962,
  'recall': 0.9722147530698216},
 'MISC': {'f1': 0.8947848285922684,
  'number': 5455,
  'precision': 0.8900779974605478,
  'recall': 0.8995417048579285},
 'ORG': {'f1': 0.9608455440787893,
  'number': 6190,
  'precision': 0.9525321479599936,
  'recall': 0.9693053311793215},
 'PER': {'f1': 0.9866095873363532,
  'number': 10042,
  'precision': 0.9863640887827212,
  'recall': 0.9868552081258714},
 'overall_accuracy': 0.9943252485807478,
 'overall_f1': 0.9605338995904747,
 'overall_precision': 0.9570210965362993,
 'overall_recall': 0.9640725855559615}

Now, can you try to evaluate the NER task performance on a multilingual dataset? with a different language model?
1. Think about what Language Model on HuggingFace can help you Fine-tune over a multilingual dataset. Hint: XLM-R
2. Go back to the commented out multilingual dataset concatetation notebook cell and try to include a few more languages in the dataset.
3. Try to fine-tune the model on this dataset.
4. Report the performance in a MARKDOWN CELL below, showing your work on multilingual model fine-tuning in the cells below. 
5. Submit this notebook to Google Classroom

## Conclusions

### For single language(English).

***** Running Evaluation *****
  Num examples = 11597
  Batch size = 16
  [725/725 01:30]
  ***
  {'epoch': 1.0, 'eval_accuracy': 0.9917501076154293,
 'eval_f1': 0.927889847882761,
 'eval_loss': 0.02557576447725296,
 'eval_precision': 0.9241706161137441,
 'eval_recall': 0.9316391359593392,
 'eval_runtime': 98.6802,
 'eval_samples_per_second': 117.521,
 'eval_steps_per_second': 7.347}

***** Running Prediction *****
  Num examples = 11597
  Batch size = 16
 [725/725 03:09] 
 ***
{'LOC': {'f1': 0.9338118687922543, 
  'number': 5955,
  'precision': 0.9283106538333886,
  'recall': 0.9393786733837112},
  ***
 'MISC': {'f1': 0.8835475326204257,
  'number': 5061,
  'precision': 0.877435697583788,
  'recall': 0.8897451096621221},
  ***
 'ORG': {'f1': 0.9315942028985505,
  'number': 3449,
  'precision': 0.9313242538394668,
  'recall': 0.931864308495216},
  ***
 'PER': {'f1': 0.9619549592716818,
  'number': 5210,
  'precision': 0.9605741626794259,
  'recall': 0.9633397312859885},
  ***
 'overall_accuracy': 0.9917501076154293,
 'overall_f1': 0.927889847882761,
 'overall_precision': 0.9241706161137441,
 'overall_recall': 0.9316391359593392}

### For multi language (es, de)

***** Running Evaluation *****
  Num examples = 21990
  Batch size = 16
 [1375/1375 04:11]
 ***
{'epoch': 1.0,
 'eval_accuracy': 0.9943252485807478,
 'eval_f1': 0.9605338995904747,
 'eval_loss': 0.01865452341735363,
 'eval_precision': 0.9570210965362993,
 'eval_recall': 0.9640725855559615,
 'eval_runtime': 267.2823,
 'eval_samples_per_second': 82.273,
 'eval_steps_per_second': 5.144}

***** Running Prediction *****
  Num examples = 21990
  Batch size = 16
 [1375/1375 08:39]***
{'LOC': {'f1': 0.9691744102930665,
  'number': 11157,
  'precision': 0.9661530239600962,
  'recall': 0.9722147530698216},
  ***
 'MISC': {'f1': 0.8947848285922684,
  'number': 5455,
  'precision': 0.8900779974605478,
  'recall': 0.8995417048579285},
  ***
 'ORG': {'f1': 0.9608455440787893,
  'number': 6190,
  'precision': 0.9525321479599936,
  'recall': 0.9693053311793215},
  ***
 'PER': {'f1': 0.9866095873363532,
  'number': 10042,
  'precision': 0.9863640887827212,
  'recall': 0.9868552081258714},
  ***
 'overall_accuracy': 0.9943252485807478,
 'overall_f1': 0.9605338995904747,
 'overall_precision': 0.9570210965362993,
 'overall_recall': 0.9640725855559615}